In [ ]:
import pandas as pd
import numpy as np
from functools import reduce
import pyodbc
from sqlalchemy import create_engine
import urllib
from datetime import date
from pathlib import Path

In [ ]:
cnxn = pyodbc.connect(server = 'bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com', 
                      driver = '{ODBC Driver 17 for SQL Server}',
                      database = 'ca_cannabis',
                      UID = 'admin',
                      PWD = 'N19lrqxnurTUJLJT6GFe')

In [ ]:
df_roll = pd.read_sql("SELECT * FROM ca_roll", cnxn)
len(df_roll)

In [ ]:
filenames = Path('../ar_reports/orig/no_neg').glob('*.csv')
list_of_dfs = [pd.read_csv(file) for file in filenames]
for dataframe, file in zip(list_of_dfs, filenames):
    Dataframe['file'] = file
df_file = pd.concat(list_of_dfs, ignore_index=True, sort=False)

In [ ]:
# df_file['date']

In [ ]:
# df_file.to_csv('../credit_score_files/df_file_10_05_20.csv', index=False)

In [ ]:
df_file['date'] = pd.to_datetime(df_file['date'], format="%m/%d/%Y")

In [ ]:
df_ar = df_file

In [ ]:
df_ar.info()

merge AR report on license number with roll_up id df

In [ ]:
df_ar['license_number'] = df_ar['license_number'].str.strip()

In [ ]:
df_ar_roll = pd.merge(df_ar, df_roll, on='license_number', how='left')
df_ar_roll.head()

number of accounts still missing roll up id after above merge

In [ ]:
len(df_ar_roll[df_ar_roll['roll_up_id'].isnull()])

split out the ones where roll up id is filled

In [ ]:
df_ar_roll_fill = df_ar_roll[~df_ar_roll['roll_up_id'].isnull()]
len(df_ar_roll_fill)

### includes companies where license number is NOT populated

In [ ]:
df_ar_roll.to_csv('../credit_score_files/combined_lists_include_nulls_2020_12_17.csv', index=False)

### only includes companies where license number IS populated

In [ ]:
df_ar_roll_fill.to_csv('../credit_score_files/combined_lists_2020_12_17.csv', index=False)

In [ ]:
df_all = df_ar_roll_fill

In [ ]:
df_all

In [ ]:
df_all.to_csv('../credit_score_files/credit_score_2020_12_21.csv', index=False)

## code for credit database stats

In [ ]:
# df_bal = df_all.groupby(by='client')['bal_out'].sum()

In [ ]:
# df_bal

In [ ]:
# df_bal.to_csv('../credit_score_files/client_balances_2020_12_17.csv', index=True)

In [ ]:
# len(df_all['license_number'].unique())

In [ ]:
# len(df_all['roll_up_id'].unique())

In [ ]:
# ar_final.groupby(['ar_account', 'date'], as_index=False)['days_over_due'].sum()

In [ ]:
# df_count = df_all.groupby('client', as_index=True)['license_number'].count()

In [ ]:
# df_nunique = df_all.groupby('client', as_index=True)['license_number'].nunique()

In [ ]:
# df_unique_roll = df_all.groupby('client', as_index=True)['roll_up_id'].nunique()

In [ ]:
# df_count.rename('license_count', inplace=True)

In [ ]:
# df_all_count = pd.concat([df_count, df_nunique, df_unique_roll], axis=1)

In [ ]:
# df_all_count

In [ ]:
# df_all_count.to_csv('../credit_score_files/df_all_count_10_05_20.csv', index=True)

In [ ]:
# df_count.head(1)

In [ ]:
# df_nunique.head(1)

In [ ]:
# df_unique_roll.head(1)

In [ ]:
# client_max_date = df_all[['client', 'date']].groupby('client', as_index=False).max()

In [ ]:
# client_max_date

In [ ]:
# len(df_all)

In [ ]:
# len(df_all['license_number'].unique())

In [ ]:
# len(df_all['roll_up_id'].unique())

In [ ]:
# df_all['bal_out'].sum()

# Credit score

determine which month has the most outstanding / count

In [ ]:
# df_all.groupby(by='date').sum()['bal_out'].apply("{:,.10}".format)

In [ ]:
# df_all.groupby(by='date').count()['license_number']

month to use for analysis

can use the below to add together all historical and then do another one based on recent month. only grab historical that match most recent month (maybe use 3/31 as it has more?)

In [ ]:
# df_rct = df_all[(df_all['date'] <= pd.Timestamp(date(2020,3,31))) & (df_all['date'] >= pd.Timestamp(date(2020,3,16)))]
# df_rct = df_all[df_all['date'] == pd.Timestamp(date(2020, 4, 10))]
# df_rct = df_all[(df_all['date'] <= pd.Timestamp(date(2020,3,31))) & (df_all['date'] >= pd.Timestamp(date(2020,3,16)))]
# df_rct = df_all_recent[df_all_recent['date'] >= pd.Timestamp(date(2020,1,1))]
# df_rct = df_all[df_all['date'] <= pd.Timestamp(date(2020, 5, 31))]

In [ ]:
# len(df_rct)

In [ ]:
# client_max_date.merge(df_all, on=['client', 'date'], how='left').sort_values(by='client', ascending=True)

In [ ]:
# df_all_recent = client_max_date.merge(df_all, on=['client', 'date'], how='left')

In [ ]:
# df_all_recent = client_max_date.merge(df_all, on=['client', 'date'], how='right')

In [ ]:
# df_all_recent.to_csv('../credit_score_files/df_all_recent_09_03_v6.csv', index=False)


In [ ]:
# df_rct.groupby('date').count()

In [ ]:
# df_rct = df_all_recent

In [ ]:
count_groupby = df_rct.groupby('roll_up_id', as_index=False)['license_number'].count()
sum_payors = df_rct.groupby('roll_up_id', as_index=False)['count_payor'].sum()
sum_days_over = df_rct.groupby('roll_up_id', as_index=False)['sum_days_over'].sum()
sum_orig_amt = df_rct.groupby('roll_up_id', as_index=False)['orig_amt'].sum()
sum_bal_out = df_rct.groupby('roll_up_id', as_index=False)['bal_out'].sum()
sum_current = df_rct.groupby('roll_up_id', as_index=False)['current'].sum()
sum_30_day = df_rct.groupby('roll_up_id', as_index=False)['30_days'].sum()
sum_60_day = df_rct.groupby('roll_up_id', as_index=False)['60_days'].sum()
sum_90_day = df_rct.groupby('roll_up_id', as_index=False)['90_days'].sum()
sum_over_90 = df_rct.groupby('roll_up_id', as_index=False)['over_90_days'].sum()
sum_delinq = df_rct.groupby('roll_up_id', as_index=False)['delinquent'].sum()
sum_sum = df_rct.groupby('roll_up_id', as_index=False)['summary'].sum()

In [ ]:
data_frames = [count_groupby, sum_payors, sum_days_over, sum_orig_amt, sum_bal_out, sum_current, sum_30_day,
               sum_60_day, sum_90_day, sum_over_90, sum_delinq, sum_sum]

In [ ]:
df_rct_sums = reduce(lambda left,right: pd.merge(left,right,on=['roll_up_id'], how='outer'), data_frames)
len(df_rct_sums)

remove if under 500

In [ ]:
df_rct_bal = df_rct_sums[df_rct_sums['bal_out'] > 500]

In [ ]:
pd.set_option('mode.chained_assignment', None)

list_curr = df_rct_bal[(df_rct_bal['current'] < 500) & (df_rct_bal['current'] > 0)].index
df_rct_bal['current'].loc[list_curr] = 0

list_30 = df_rct_bal[(df_rct_bal['30_days'] < 500) & (df_rct_bal['30_days'] > 0)].index
df_rct_bal['30_days'].loc[list_30] = 0

list_60 = df_rct_bal[(df_rct_bal['60_days'] < 500) & (df_rct_bal['60_days'] > 0)].index
df_rct_bal['60_days'].loc[list_60] = 0

list_90 = df_rct_bal[(df_rct_bal['90_days'] < 500) & (df_rct_bal['90_days'] > 0)].index
df_rct_bal['90_days'].loc[list_90] = 0

list_ovr_90 = df_rct_bal[(df_rct_bal['over_90_days'] < 500) & (df_rct_bal['over_90_days'] > 0)].index
df_rct_bal['over_90_days'].loc[list_ovr_90] = 0

In [ ]:
df_rct_bal.drop(columns='bal_out', inplace=True)

recalculate the balance outstanding

In [ ]:
pd.set_option('mode.chained_assignment', None)

df_rct_bal['bal_out'] = df_rct_bal['current'] + df_rct_bal['30_days'] + df_rct_bal['60_days'] + df_rct_bal['90_days'] + df_rct_bal['over_90_days']

In [ ]:
df_rct_bal = df_rct_bal[['roll_up_id', 'bal_out', 'current', '30_days', '60_days', '90_days', 'over_90_days', 'delinquent']]

In [ ]:
# df_rct_bal[df_rct_bal['bal_out'] == 0]

In [ ]:
# len(df_rct_bal)

In [ ]:
# len(df_rct_bal[df_rct_bal['bal_out'] != 0])

In [ ]:
# df_rct_bal = df_rct_bal[df_rct_bal['bal_out'] != 0]

In [ ]:
# df_rct_bal[df_rct_bal['times_total'] < 0]

count if DPD over 0

In [ ]:
pd.set_option('mode.chained_assignment', None)

df_rct_bal['count_curr'] = df_rct_bal['current'].apply(lambda x: 0 if x == 0 else 1)
df_rct_bal['count_30'] = df_rct_bal['30_days'].apply(lambda x: 0 if x == 0 else 1)
df_rct_bal['count_60'] = df_rct_bal['60_days'].apply(lambda x: 0 if x == 0 else 1)
df_rct_bal['count_90'] = df_rct_bal['90_days'].apply(lambda x: 0 if x == 0 else 1)
df_rct_bal['count_ovr_90'] = df_rct_bal['over_90_days'].apply(lambda x: 0 if x == 0 else 1)

apply a score for each count

In [ ]:
df_rct_bal['score_curr'] = df_rct_bal['count_curr'].apply(lambda x: 0 if x == 0 else 2)
df_rct_bal['score_30'] = df_rct_bal['count_30'].apply(lambda x: 0 if x == 0 else 4)
df_rct_bal['score_60'] = df_rct_bal['count_60'].apply(lambda x: 0 if x == 0 else 8)
df_rct_bal['score_90'] = df_rct_bal['count_90'].apply(lambda x: 0 if x == 0 else 16)
df_rct_bal['score_ovr_90'] = df_rct_bal['count_ovr_90'].apply(lambda x: 0 if x == 0 else 32)

In [ ]:
df_rct_bal['score_total'] = df_rct_bal['score_curr'] + df_rct_bal['score_30'] + df_rct_bal['score_60'] + df_rct_bal['score_90'] + df_rct_bal['score_ovr_90']

% of DPD over balance outstanding

In [ ]:
df_rct_bal['pct_curr'] = df_rct_bal['current'] / df_rct_bal['bal_out']
df_rct_bal['pct_30'] = df_rct_bal['30_days'] / df_rct_bal['bal_out']
df_rct_bal['pct_60'] = df_rct_bal['60_days'] / df_rct_bal['bal_out']
df_rct_bal['pct_90'] = df_rct_bal['90_days'] / df_rct_bal['bal_out']
df_rct_bal['pct_ovr_90'] = df_rct_bal['over_90_days'] / df_rct_bal['bal_out']

multiply score by % of DPD over outstanding

In [ ]:
df_rct_bal['times_curr'] = df_rct_bal['score_curr'] * df_rct_bal['pct_curr']
df_rct_bal['times_30'] = df_rct_bal['score_30'] * df_rct_bal['pct_30']
df_rct_bal['times_60'] = df_rct_bal['score_60'] * df_rct_bal['pct_60']
df_rct_bal['times_90'] = df_rct_bal['score_90'] * df_rct_bal['pct_90']
df_rct_bal['times_ovr_90'] = df_rct_bal['score_ovr_90'] * df_rct_bal['pct_ovr_90']

In [ ]:
df_rct_bal['times_total'] = df_rct_bal['times_curr'] + df_rct_bal['times_30'] + df_rct_bal['times_60'] + df_rct_bal['times_90'] + df_rct_bal['times_ovr_90'] 

get unique id from roll up

In [ ]:
df_roll_edit = df_roll[['roll_up_id', 'company_roll_up']]

In [ ]:
pd.set_option('mode.chained_assignment', None)

df_roll_edit.drop_duplicates(inplace=True)

In [ ]:
df_final = df_rct_bal.merge(df_roll_edit, on='roll_up_id')

In [ ]:
df_final['score_total'].quantile([0, 0.25, 0.5, 0.75, 1])

In [ ]:
df_final['times_total'].quantile([0, 0.25, 0.5, 0.75, 1])

In [ ]:
df_final['quant_score'] = pd.qcut(df_final['score_total'], 4, labels=False)

In [ ]:
df_final['quant_times'] = pd.qcut(df_final['times_total'], 4, labels=False)

In [ ]:
df_final = df_final[['roll_up_id', 'company_roll_up', 'bal_out', 'current', '30_days', '60_days', '90_days',
       'over_90_days', 'delinquent', 
        'count_curr', 'count_30', 'count_60','count_90', 'count_ovr_90', 
        'score_curr', 'score_30', 'score_60','score_90', 'score_ovr_90', 'score_total', 
                     'quant_score', 
        'pct_curr', 'pct_30', 'pct_60', 'pct_90', 'pct_ovr_90', 
        'times_curr', 'times_30', 'times_60','times_90', 'times_ovr_90', 'times_total','quant_times']]

In [ ]:
# df_final[df_final['roll_up_id'] == 3029]

In [ ]:
# df_final[df_final['company_roll_up'] == 'haute supply']

In [ ]:
df_final.to_csv('../credit_score_files/credit_score_2020_09_03_v6.csv', index=False)

In [ ]:
df_2020_Q2 = df_final

In [ ]:
df_2020_Q1 = df_final

In [ ]:
df_2019_Q4 = df_final

In [ ]:
# df_2019_09_30 = df_final

In [ ]:
df_2019_Q3 = df_2019_09_30

In [ ]:
# df_2019_06_30 = df_final

In [ ]:
df_2019_Q2 = df_2019_06_30

In [ ]:
# df_2019_03_31 = df_final

In [ ]:
df_2019_Q1 = df_2019_03_31

In [ ]:
# df_2019_06_30

In [ ]:
# len(df_final)
#372

In [ ]:
# df_final[df_final['delinquent'] > 0]

In [ ]:
df_2019_Q1.columns

In [ ]:
df_2019_Q1_edit = df_2019_Q1[['roll_up_id', 
                            'company_roll_up', 
                            'bal_out', 'current', '30_days',
                            '60_days', '90_days', 'over_90_days', 'delinquent', 
                            'score_total','quant_score', 
                            'times_total', 'quant_times']]

## roll up analysis historical

In [ ]:
count_groupby = df_all.groupby('roll_up_id', as_index=False)['license_number'].count()

In [ ]:
sum_payors = df_all.groupby('roll_up_id', as_index=False)['count_payor'].sum()
sum_days_over = df_all.groupby('roll_up_id', as_index=False)['sum_days_over'].sum()
sum_orig_amt = df_all.groupby('roll_up_id', as_index=False)['orig_amt'].sum()
sum_bal_out = df_all.groupby('roll_up_id', as_index=False)['bal_out'].sum()
sum_current = df_all.groupby('roll_up_id', as_index=False)['current'].sum()
sum_30_day = df_all.groupby('roll_up_id', as_index=False)['30_days'].sum()
sum_60_day = df_all.groupby('roll_up_id', as_index=False)['60_days'].sum()
sum_90_day = df_all.groupby('roll_up_id', as_index=False)['90_days'].sum()
sum_over_90 = df_all.groupby('roll_up_id', as_index=False)['over_90_days'].sum()
sum_delinq = df_all.groupby('roll_up_id', as_index=False)['delinquent'].sum()
sum_sum = df_all.groupby('roll_up_id', as_index=False)['summary'].sum()

In [ ]:
# count_groupby

In [ ]:
data_frames = [count_groupby, sum_payors, sum_days_over, sum_orig_amt, sum_bal_out, sum_current, sum_30_day,
               sum_60_day, sum_90_day, sum_over_90, sum_delinq, sum_sum]

In [ ]:
df_sums = reduce(lambda left,right: pd.merge(left,right,on=['roll_up_id'], how='left'), data_frames)

In [ ]:
df_sums['pct_curr'] = df_sums['current'] / df_sums['bal_out']

In [ ]:
df_sums['pct_30'] = df_sums['30_days'] / df_sums['bal_out']
df_sums['pct_60'] = df_sums['60_days'] / df_sums['bal_out']
df_sums['pct_90'] = df_sums['90_days'] / df_sums['bal_out']
df_sums['pct_ovr_90'] = df_sums['over_90_days'] / df_sums['bal_out']

In [ ]:
df_sums.fillna(value=0, inplace=True)

In [ ]:
df_sums.head(1)

In [ ]:
# df_sums[df_sums['bal_out'] == 0]

In [ ]:
df_sums_bal = df_sums[df_sums['bal_out'] > 0]

In [ ]:
len(df_sums_bal)

In [ ]:
# df_sums_bal['orig_amt'].quantile([0.1,0.2,0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [ ]:
df_sums_bal['bal_out'].quantile([0.1,0.2,0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [ ]:
# df_sums_bal['bal_quant'] = pd.qcut(df_sums_bal['bal_out'], 10, labels=False)

In [ ]:
# df_sums_bal.groupby('bal_quant').count()

In [ ]:
# pd.qcut(df_sums_bal['pct_curr'], 3, labels=False)
# pd.qcut(df_sums_bal['pct_30'], 3, labels=False)
# pd.qcut(df_sums_bal['sum_60_quant'], 4, labels=False)
# pd.qcut(df_sums_bal['sum_90'], 4, labels=False)
# pd.qcut(df_sums_bal['sum_ovr_90'], 2, labels=False)

In [ ]:
df_sums_bal.head(1)

In [ ]:
df_sums_bal.columns

In [ ]:
len(df_sums_bal.columns)

In [ ]:
df_sums_bal.iloc[:2, 22:27]

In [ ]:
# df_sums_bal.iloc[:,14:19].apply(lambda x: pd.Series([(x < 0).sum()]), axis=1)
# train_test.iloc[:,[6,15]].apply(lambda x: 
#                           0 if (x.loc['past_due_60'] > 95) 
#                           else x, axis=1)

df_sums_bal['count_curr'] = df_sums_bal.iloc[:,6].apply(lambda x: 0 if x == 0 else 1)
df_sums_bal['count_30'] = df_sums_bal.iloc[:,7].apply(lambda x: 0 if x == 0 else 1)
df_sums_bal['count_60'] = df_sums_bal.iloc[:,8].apply(lambda x: 0 if x == 0 else 1)
df_sums_bal['count_90'] = df_sums_bal.iloc[:,9].apply(lambda x: 0 if x == 0 else 1)
df_sums_bal['count_ovr_90'] = df_sums_bal.iloc[:,10].apply(lambda x: 0 if x == 0 else 1)

In [ ]:
df_sums_bal['score_curr'] = df_sums_bal.iloc[:,22].apply(lambda x: 0 if x == 0 else 2)
df_sums_bal['score_30'] = df_sums_bal.iloc[:,23].apply(lambda x: 0 if x == 0 else 4)
df_sums_bal['score_60'] = df_sums_bal.iloc[:,24].apply(lambda x: 0 if x == 0 else 6)
df_sums_bal['score_90'] = df_sums_bal.iloc[:,25].apply(lambda x: 0 if x == 0 else 8)
df_sums_bal['score_ovr_90'] = df_sums_bal.iloc[:,26].apply(lambda x: 0 if x == 0 else 10)

In [ ]:
# df_sums_bal.drop(columns=['count_curr', 'count_30', 'count_60', 'count_90', 'count_ovr_90', 'score_curr', 'score_30', 'score_60', 'score_90', 'score_ovr_90'], inplace=True)

In [ ]:
# df_sums_bal.drop(columns='score_total', inplace=True)

In [ ]:
# df_sums_bal.iloc[:,22]

In [ ]:
df_sums_bal.iloc[:5, 6:11]

In [ ]:
df_sums_bal.iloc[:5, 22:27]

In [ ]:
df_sums_bal.to_csv('../ar_reports/df_sums_bal_04_29.csv', index=False)

In [ ]:
df_sums_bal.iloc[:20, 6:11]

In [ ]:
df_sums_bal.iloc[:20, 22:33]

In [ ]:
df_feb_bal['score_total'].quantile([0.25, 0.5, 0.75])

In [ ]:
df_sums_bal['score_total'].quantile(q=[0.25, 0.5, 0.75])

In [ ]:
df_sums_bal['score_total'] = df_sums_bal['score_curr'] + df_sums_bal['score_30'] + df_sums_bal['score_60'] + df_sums_bal['score_90'] + df_sums_bal['score_ovr_90']

In [ ]:
pd.qcut(df_sums_bal['score_total'], 4, labels=False)

In [ ]:
df_sums_bal.head(1)

In [ ]:
df_sums_bal['sum_30'] = df_sums_bal['pct_curr'] + df_sums_bal['pct_30']
df_sums_bal['sum_60'] = df_sums_bal['pct_curr'] + df_sums_bal['pct_30'] + df_sums_bal['pct_60']
df_sums_bal['sum_90'] = df_sums_bal['pct_curr'] + df_sums_bal['pct_30'] + df_sums_bal['pct_60'] + df_sums_bal['pct_90']
df_sums_bal['sum_ovr_90'] = df_sums_bal['pct_curr'] + df_sums_bal['pct_30'] + df_sums_bal['pct_60'] + df_sums_bal['pct_90'] + df_sums_bal['pct_ovr_90']

In [ ]:
# df_sums_bal.drop(columns='sum_60', inplace=True)

In [ ]:
# df_sums_bal.rename(columns={'sum_60_quant': 'sum_60'}, inplace=True)

In [ ]:
df_sums_bal['sum_ovr_90'].median()

In [ ]:
df_sums_bal['over_90_days'].median()

In [ ]:
# pd.qcut(df_sums_bal['60_days'], 2, labels=False)
# pd.qcut(df_sums_bal['60_days'], 1, labels=False)
# df_sums_bal['curr_quant'] = pd.qcut(df_sums_bal['current'], 2, labels=False)
# df_sums_bal['30_quant'] = pd.qcut(df_sums_bal['30_days'], 2, labels=False)

In [ ]:
# pd.set_option('mode.chained_assignment', None)

# # df_sums_bal['bal_quant'] = pd.qcut(df_sums_bal['bal_out'], 10, labels=False)
# # df_sums_bal['curr_quant'] = pd.qcut(df_sums_bal['current'], 10, labels=False)
# # df_sums_bal['30_quant'] = pd.qcut(df_sums_bal['30_days'], 10, labels=False)
# df_sums_bal['60_quant'] = pd.qcut(df_sums_bal['60_days'], 10, labels=False)
# df_sums_bal['90_quant'] = pd.qcut(df_sums_bal['90_days'], 10, labels=False)
# df_sums_bal['ovr_90_quant'] = pd.qcut(df_sums_bal['over_90_days'], 10, labels=False)

In [ ]:
# for i in range(len(df_sums_bal['bal_out'])):
#     print(df_sums_bal['bal_out'].iloc[i])

In [ ]:
df_sums_bal['pct_curr'].qcut()

In [ ]:
# df_sums_bal['over_90_days'].quantile(q=[.25, .5, .75])
df_sums_bal['over_90_days'].quantile(q=[0,.5,1])

In [ ]:
df_sums_bal['over_90_days'].quantile([0.1,0.2,0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [ ]:
df_sums_bal['pct_30'].quantile([0.1,0.2,0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [ ]:
df_sums_bal['pct_60'].quantile([0.1,0.2,0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [ ]:
df_sums_bal['pct_90'].quantile([0.1,0.2,0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [ ]:
df_sums_bal['pct_ovr_90'].quantile([0.1,0.2,0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [ ]:
# df_all.groupby(by='roll_up_id').sum()

In [ ]:
df_all_group_sum = df_all.groupby(by='roll_up_id').sum()

In [ ]:
# df_all.rename(columns={'license provided': 'license_provided'}, inplace=True)

In [ ]:
# df_all_group_sum.rename(columns={'license provided': 'license_provided'}, inplace=True)

In [ ]:
df_group = df_all_group_sum[['license_provided', 'count_payor', 'sum_days_over', 'orig_amt', 'bal_out',
                 'current', '30_days', '60_days', '90_days', 'over_90_days', 'delinquent', 'summary']]

In [ ]:
df_group_bal = df_group[df_group['bal_out'] > 0]

In [ ]:
df_roll.head()

In [ ]:
df_roll_cut = df_roll[['roll_up_id', 'company_roll_up']]

In [ ]:
df_roll_cut = df_roll_cut.drop_duplicates()

In [ ]:
df_group_bal.merge(df_roll_cut, on='roll_up_id')

In [ ]:
# df.time_diff.quantile([0.25,0.5,0.75])
df_group_bal['30_days'].quantile([0.25,0.5,0.75])

In [ ]:
df_all['60_days'].quantile([0.25,0.5,0.75])

In [ ]:
df_all['90_days'].quantile([0.25,0.5,0.75])

In [ ]:
# df_all.to_csv('../ar_reports/kiva_sql_combined.csv', index=False)

In [ ]:
# df_all = pd.read_csv('../ar_reports/kiva_sql_combined.csv')

In [ ]:
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

In [ ]:
9/413

In [ ]:
df_all['delinquent'].sum()

In [ ]:
y = df_all['delinquent']

In [ ]:
# need to split out the columns that are numeric to use for train / test split

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

new_df = df_all.select_dtypes(include=numerics)

In [ ]:
new_df.columns

In [ ]:
new_df.drop(columns=['delinquent', 'roll_up_id', 'in_db', 'contact_phone', 'contact_zip'], inplace=True)

In [ ]:
new_df.fillna(value=0, inplace=True)

In [ ]:
# df_all.info()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(new_df, y, test_size=0.2)

In [ ]:
lm = linear_model.LinearRegression()

In [ ]:
model = lm.fit(X_train, y_train)

In [ ]:
predictions = lm.predict(X_test)

In [ ]:
model.score(X_test, y_test)

In [ ]:
len(new_df)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(train.corr(), annot=True)

In [ ]:
train_corr = train.corr()
plt.figure(figsize=(10,10))
sns.heatmap(train_corr[['SeriousDlqin2yrs']].sort_values(by=['SeriousDlqin2yrs'], ascending=False), vmin=-1, vmax=1, annot=True);

In [ ]:
plt.hist(train['NumberOfTimes90DaysLate']);

In [ ]:
train.groupby('NumberOfTimes90DaysLate').count()

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn import linear_model

In [ ]:
# df_ar = pd.read_sql("SELECT * FROM ar_report", cnxn)
# len(df_ar)

# ar.to_csv('../ar_reports/humboldt_ar_clean_04_23_20.csv', index=False)

In [ ]:
# df_main = pd.read_sql("SELECT * FROM ca_main", cnxn)
# len(df_main)

# df_contact = pd.read_sql("SELECT * FROM ca_contact", cnxn)
# len(df_contact)

In [ ]:
# df_ar_main = pd.merge(df_ar_roll_fill, df_main, on='license_number', how='left')
# df_all = pd.merge(df_ar_main, df_contact, on='license_number', how='left')

# df_all.to_csv('../ar_reports/ar_kiva_hf_sql_detail_v2.csv', index=False)

# df_all = pd.read_csv('../ar_reports/ar_kiva_hf_sql_detail_v2.csv')

# df_all[['current']]
# df_summary = df_all[df_all['summary'] == 0]
# df_sum_0 = df_summary[df_summary['bal_out'] > 0]

In [ ]:
# import seaborn as sns
# df_sum_0[df_sum_0['current'] > 0]['current']
# sns.distplot(df_sum_0['current'])
# import matplotlib.pyplot as plt
# df_sum_0['current'].
# format(df_sum_0[df_sum_0['current'] > 0]['current'], ",")

In [ ]:
# plt.figure(figsize=(12,6))
# x = df_sum_0[df_sum_0['current'] > 0]['current']
# ax = sns.distplot(x)

# # ax.get_xaxis().set_major_formatter(plt.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

# ax.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

# xlabels = ['{:,.2f}'.format(x) for x in g.get_xticks()/1000]
# g.set_xticklabels(xlabels)

# sns.set(); np.random.seed(0)
# x = np.random.randn(100)
# ax = sns.distplot(x)

In [ ]:
# df_ar_roll['license_number'].iloc[1]
# df_ar_roll['license_number'].str.strip()
# df_ar_roll[df_ar_roll['roll_up_id'].isnull()]

In [ ]:
# df_all['30_days'].sum()
# df_all['current'].quantile([0.25,0.5,0.75])
# df_all['current'].median()
# df_all.info()

In [ ]:
# df_all.to_csv('../ar_reports/ar_combined_04_29.csv', index=False)
# df_ar = pd.read_csv('../ar_reports/kiva_hr_ar_combined.csv')

In [ ]:
# df_feb_bal.rename(columns={'total':'bal_out'}, inplace=True)
# df_feb_bal[df_feb_bal['bal_out'] != df_feb_bal['total']]
# df_feb_bal['current'].apply(lambda x: 0 if x == 0 else 1)

In [ ]:
# df_rct_bal.columns.get_loc("times_total")

In [ ]:
# df_rct_sums[df_rct_sums['delinquent'] > 0]
# df_feb_bal = df_feb_sums[df_feb_sums['bal_out'] > 0]

In [ ]:
# df_apex = pd.read_csv('../ar_reports/orig/no_neg/apex_ar_no_neg.csv')
# df_apex['date'] = pd.to_datetime(df_apex['date'], format="%m/%d/%Y")

# df_big_petes = pd.read_csv('../ar_reports/orig/no_neg/big_petes_ar_no_neg.csv')
# df_big_petes['date'] = pd.to_datetime(df_big_petes['date'], format="%m/%d/%Y")

# df_biscotti = pd.read_csv('../ar_reports/orig/no_neg/biscotti_ar_no_neg.csv')
# df_biscotti['date'] = pd.to_datetime(df_biscotti['date'], format="%Y/%m/%d")

# df_buddies = pd.read_csv('../ar_reports/orig/no_neg/buddies_ar_no_neg.csv')
# df_buddies['date'] = pd.to_datetime(df_buddies['date'], format="%m/%d/%Y")

# df_desert_1 = pd.read_csv('../ar_reports/orig/no_neg/desert_road_sum_08_12_19.csv')
# df_desert_1['date'] = pd.to_datetime(df_desert_1['date'], format="%m/%d/%Y")

# df_desert_2 = pd.read_csv('../ar_reports/orig/no_neg/desert_road_sum_12_31_19.csv')
# df_desert_2['date'] = pd.to_datetime(df_desert_2['date'], format="%m/%d/%Y")

# df_desert_3 = pd.read_csv('../ar_reports/orig/no_neg/desert_road_sum_02_29_20.csv')
# df_desert_3['date'] = pd.to_datetime(df_desert_3['date'], format="%m/%d/%Y")

# df_falcon = pd.read_csv('../ar_reports/orig/no_neg/falcon_ar_no_neg.csv')
# df_falcon['date'] = pd.to_datetime(df_falcon['date'], format="%m/%d/%Y")

# df_headwaters = pd.read_csv('../ar_reports/orig/no_neg/headwaters_ar_no_neg.csv')
# df_headwaters['date'] = pd.to_datetime(df_headwaters['date'], format="%m/%d/%Y")

# df_flowerhired = pd.read_csv('../ar_reports/orig/no_neg/flowerhired_ar_no_neg.csv')
# df_flowerhired['date'] = pd.to_datetime(df_flowerhired['date'], format="%m/%d/%Y")

# df_guild = pd.read_csv('../ar_reports/orig/no_neg/guild_extracts_ar_no_neg.csv')
# df_guild['date'] = pd.to_datetime(df_guild['date'], format="%m/%d/%Y")

# df_henrys = pd.read_csv('../ar_reports/orig/no_neg/henrys_ar_no_neg.csv')
# df_henrys['date'] = pd.to_datetime(df_henrys['date'], format="%m/%d/%Y")

# df_hf = pd.read_csv('../ar_reports/orig/no_neg/humboldt_ar_no_neg_04_29.csv')
# df_hf['date'] = pd.to_datetime(df_hf['date'], format="%m/%d/%Y")

# df_kv = pd.read_csv('../ar_reports/orig/no_neg/kiva_ar_no_neg_combined.csv')
# df_kv['date'] = pd.to_datetime(df_kv['date'], format="%m/%d/%Y")

# df_lob_det = pd.read_csv('../ar_reports/orig/no_neg/LOB_ar_detail_no_neg_3_31_20.csv')
# df_lob_det['date'] = pd.to_datetime(df_lob_det['date'], format="%m/%d/%Y")

# df_lob_sum = pd.read_csv('../ar_reports/orig/no_neg/LOB_summary_ar_no_neg.csv')
# df_lob_sum['date'] = pd.to_datetime(df_lob_sum['date'], format="%m/%d/%Y")

# df_nb= pd.read_csv('../ar_reports/orig/no_neg/nabis_ar_no_neg_05_13.csv')
# df_nb['date'] = pd.to_datetime(df_nb['date'], format="%m/%d/%Y")

# df_sherb= pd.read_csv('../ar_reports/orig/no_neg/sherbinkskis_ar_no_neg.csv')
# df_sherb['date'] = pd.to_datetime(df_sherb['date'], format="%m/%d/%Y")

# df_shryne= pd.read_csv('../ar_reports/orig/no_neg/shryne_ar_no_neg.csv')
# df_shryne['date'] = pd.to_datetime(df_shryne['date'], format="%m/%d/%Y")

# df_thirty_one= pd.read_csv('../ar_reports/orig/no_neg/thirty_one_labs_ar_no_neg.csv')
# df_thirty_one['date'] = pd.to_datetime(df_thirty_one['date'], format="%m/%d/%Y")

In [ ]:
# df_ar = df_apex.append(df_big_petes)
# df_ar = df_ar.append(df_biscotti)
# df_ar = df_ar.append(df_buddies)
# df_ar = df_ar.append(df_desert_1)
# df_ar = df_ar.append(df_desert_2)
# df_ar = df_ar.append(df_desert_3)
# df_ar = df_ar.append(df_headwaters)
# df_ar = df_ar.append(df_flowerhired)
# df_ar = df_ar.append(df_guild)
# df_ar = df_ar.append(df_henrys)
# df_ar = df_ar.append(df_hf)
# df_ar = df_ar.append(df_kv)
# df_ar = df_ar.append(df_lob_det)
# df_ar = df_ar.append(df_lob_sum)
# df_ar = df_ar.append(df_nb)
# df_ar = df_ar.append(df_sherb)
# df_ar = df_ar.append(df_shryne)
# df_ar = df_ar.append(df_thirty_one)

In [ ]:
# len(df_all[df_all['client'] == 'Apex']['license_number'].unique())
# len(df_all[df_all['client'] == 'Apex']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'Big Petes'])
# len(df_all[df_all['client'] == 'Big Petes']['license_number'].unique())
# len(df_all[df_all['client'] == 'Big Petes']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'Beezle'])
# len(df_all[df_all['client'] == 'Beezle']['license_number'].unique())
# len(df_all[df_all['client'] == 'Beezle']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'Biscotti'])
# len(df_all[df_all['client'] == 'Biscotti']['license_number'].unique())
# len(df_all[df_all['client'] == 'Biscotti']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'Buddies']['license_number'].unique())
# len(df_all[df_all['client'] == 'Buddies']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'Chemistry'])
# len(df_all[df_all['client'] == 'Chemistry']['license_number'].unique())
# len(df_all[df_all['client'] == 'Chemistry']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'Desert Road']['license_number'].unique())
# len(df_all[df_all['client'] == 'Desert Road']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'DomPen'])
# len(df_all[df_all['client'] == 'DomPen']['license_number'].unique())
# len(df_all[df_all['client'] == 'DomPen']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'Falcon'])
# len(df_all[df_all['client'] == 'Falcon']['license_number'].unique())
# len(df_all[df_all['client'] == 'Falcon']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'Flowerhired'])
# len(df_all[df_all['client'] == 'Flowerhired']['license_number'].unique())
# len(df_all[df_all['client'] == 'Flowerhired']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'Guild Extracts'])
# len(df_all[df_all['client'] == 'Guild Extracts']['license_number'].unique())
# len(df_all[df_all['client'] == 'Guild Extracts']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'Headwaters']['license_number'].unique())
# len(df_all[df_all['client'] == 'Headwaters']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'Henrys']['license_number'].unique())
# len(df_all[df_all['client'] == 'Henrys']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'Humboldt Farms']['license_number'].unique())
# len(df_all[df_all['client'] == 'Humboldt Farms']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'Kiva']['license_number'].unique())
# len(df_all[df_all['client'] == 'Kiva']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'Left Coast Ventures'])
# len(df_all[df_all['client'] == 'Left Coast Ventures']['license_number'].unique())
# len(df_all[df_all['client'] == 'Left Coast Ventures']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'LOB']['license_number'].unique())
# len(df_all[df_all['client'] == 'LOB']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'Leune'])
# len(df_all[df_all['client'] == 'Leune']['license_number'].unique())
# len(df_all[df_all['client'] == 'Leune']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'Nabis']['license_number'].unique())
# len(df_all[df_all['client'] == 'Nabis']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'Sherbinkskis'])
# len(df_all[df_all['client'] == 'Sherbinkskis']['license_number'].unique())
# len(df_all[df_all['client'] == 'Sherbinkskis']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'Shryne']['license_number'].unique())
# len(df_all[df_all['client'] == 'Shryne']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'Thirty One Labs']['license_number'].unique())
# len(df_all[df_all['client'] == 'Thirty One Labs']['roll_up_id'].unique())
# len(df_all[df_all['client'] == 'Tri-State Distribution'])
# len(df_all[df_all['client'] == 'Tri-State Distribution']['license_number'].unique())
# len(df_all[df_all['client'] == 'Tri-State Distribution']['roll_up_id'].unique())

In [ ]:
# df_all[df_all['client'] == 'Apex']['date'].unique()

# df_all[(df_all['date'] == pd.Timestamp(date(2020, 3, 31))) & (df_all['client']== 'Apex')]['bal_out'].sum()

# df_all[(df_all['date'] == pd.Timestamp(date(2020, 4, 27))) & (df_all['client']== 'Beezle')]['bal_out'].sum()l

# df_all[df_all['client'] == 'Big Petes']['date'].unique()

# df_all[(df_all['date'] == pd.Timestamp(date(2019, 9, 30))) & (df_all['client']== 'Big Petes')]['bal_out'].sum()
# df_all[df_all['client'] == 'Biscotti']['date'].unique()
# df_all[df_all['client']== 'Biscotti']['bal_out'].sum()
# df_all[df_all['client'] == 'Buddies']['date'].unique()
# df_all[(df_all['date'] == pd.Timestamp(date(2020, 5, 31))) & (df_all['client']== 'Buddies')]['bal_out'].sum()
# df_all[df_all['client']== 'Chemistry']['bal_out'].sum()
# df_all[df_all['client']== 'DomPen']['bal_out'].sum()
# df_all[df_all['client'] == 'Desert Road']['date'].unique()
# df_all[(df_all['date'] == pd.Timestamp(date(2020, 2, 29))) & (df_all['client']== 'Desert Road')]['bal_out'].sum()
# df_all[df_all['client'] == 'Flowerhired']['date'].unique()
# df_all[(df_all['date'] == pd.Timestamp(date(2020, 6, 30))) & (df_all['client']== 'Flowerhired')]['bal_out'].sum()
# df_all[df_all['client'] == 'Guild Extracts']['date'].unique()
# df_all[(df_all['date'] == pd.Timestamp(date(2020, 3, 31))) & (df_all['client']== 'Guild Extracts')]['bal_out'].sum()
# df_all[df_all['client'] == 'Headwaters']['date'].unique()
# df_all[(df_all['date'] == pd.Timestamp(date(2020, 6, 30))) & (df_all['client']== 'Headwaters')]['bal_out'].sum()
# df_all[df_all['client'] == 'Henrys']['date'].unique()
# df_all[(df_all['date'] == pd.Timestamp(date(2019, 11, 11))) & (df_all['client']== 'Henrys')]['bal_out'].sum()
# df_all[df_all['client'] == 'Humboldt Farms']['date'].unique()
# df_all[(df_all['date'] == pd.Timestamp(date(2020, 3, 31))) & (df_all['client']== 'Humboldt Farms')]['bal_out'].sum()
# df_all[df_all['client'] == 'Kiva']['date'].unique()
# df_all[(df_all['date'] == pd.Timestamp(date(2020, 2, 25))) & (df_all['client']== 'Kiva')]['bal_out'].sum()
# df_all[df_all['client']== 'Left Coast Ventures']['bal_out'].sum()
# df_all[(df_all['date'] == pd.Timestamp(date(2019, 9, 30))) & (df_all['client']== 'Left Coast Ventures')]['bal_out'].sum()
# df_all[df_all['client']== 'Leune']['bal_out'].sum()
# df_all[df_all['client']== 'Tri-State Distribution']['bal_out'].sum()
# df_all[(df_all['date'] == pd.Timestamp(date(2020, 4, 10))) & (df_all['client']== 'LOB')]['bal_out'].sum()
# df_all[(df_all['date'] == pd.Timestamp(date(2020, 3, 31))) & (df_all['client']== 'Nabis')]['bal_out'].sum()
# df_all[df_all['client'] == 'Sherbinkskis']['date'].unique()
# df_all[df_all['client'] == 'Shryne']['date'].unique()
# df_all[(df_all['date'] == pd.Timestamp(date(2019, 1, 29))) & (df_all['client']== 'Shryne')]['bal_out'].sum()
# df_all[df_all['client'] == 'Thirty One Labs']['date'].unique()
# df_all[(df_all['date'] == pd.Timestamp(date(2020, 6, 30))) & (df_all['client']== 'Thirty One Labs')]['bal_out'].sum()

In [ ]:
# df[[‘InvoiceDate’, ‘Quantity’]].groupby(df[‘InvoiceDate’].dt.date).sum().sort_values(by=’Quantity’, ascending=False).head()

# df_all[['client', 'date', 'bal_out']].groupby(df_all['date'].dt.date).sum().sort_values(by='bal_out', ascending=False).head()

# df_all[['client', 'date', 'bal_out']].groupby('client', as_index=False).max().sort_values(by='bal_out', ascending=False)


In [ ]:
# len(df_ar_roll[df_ar_roll['delinquent'] == 1])
# df_ar_roll[df_ar_roll['license_number'] == '0']
# df_ar_roll[(df_ar_roll['roll_up_id'].isnull()) & (df_ar_roll['license_number'] == '0')]
# df_ar_roll[df_ar_roll['license_number'] == '0']
# df_ar_roll[(df_ar_roll['roll_up_id'].isnull()) & (df_ar_roll['license_number'] != '0')]
# df_ar_roll.to_csv('../credit_score_files/df_ar_roll_all.csv', index=False)